In [1]:
import warnings
warnings.filterwarnings('ignore')
import codecs
import base64
import hashlib
import ipywidgets as widgets
import time
from typing import Callable
from os.path import isfile, join
from IPython.display import HTML, display, clear_output

In [ ]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [18]:
output = widgets.Output()
list1 = ['F00000 baisser.png', 'F00001 abandonner.png']
list2 = ['F00002 abattre.png', 'F00003 abluer.png']
graph_folder = 'graph_visualization'
counter = 0

model_file = open(join(graph_folder, list1[counter]), 'rb')
model_image = widgets.Image(value=model_file.read(), width='300', height='300')
model_label = widgets.Label(value=list1[counter],style={'description_width': 'initial'})

question_file = open(join(graph_folder, list2[counter]), 'rb')
question_image = widgets.Image(value=question_file.read(), width='300', height='300')
question_label = widgets.Label(value=list2[counter],style={'description_width': 'initial'})
                       
def on_button_next_clicked(event):
    global counter
    counter += 1
    model_file = open(join(graph_folder, list1[counter]), 'rb')
    model_image.value = model_file.read()
    model_label.value = list1[counter]
    question_file = open(join(graph_folder, list2[counter]), 'rb')
    question_image.value = question_file.read()
    question_label.value = list2[counter]
                       
button_next = widgets.Button(description='Next', tooltip='Next question', style={'description_width': 'initial'})
button_next.on_click(on_button_next_clicked)

In [19]:
vbox_model = widgets.VBox([model_label, model_image])
vbox_question = widgets.VBox([question_label, question_image, button_next])
page = widgets.HBox([vbox_model, vbox_question])
display(page)